In [1]:
pip install tensorflow-addons

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
num_classes = 10
input_shape = (32, 32, 3)

AUTO=tf.data.AUTOTUNE
learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05



# Load the train and test data splits
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Display shapes of train and test datasets
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

test_dataset=(tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .shuffle(1024)
    .batch(batch_size)
    .prefetch(AUTO)
)




x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [3]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)


In [4]:
def create_encoder(backbone=keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )):
    

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = backbone(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()



Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [5]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model


In [6]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model


## model definition with the api

In [9]:

def apply_resnet_block(x,downsample,conv_by_block):
    
    depth_input=x.shape[-1]
    
    
    if downsample:
        depth=depth_input*2
        skiped=layers.Conv2D(depth,1,strides=(2,2),activation=None)(x)#linear projection
        x=layers.Conv2D(depth,3,strides=(2,2), activation='relu',padding="same")(x)
    else:
        depth=depth_input
        skiped=x
        x=layers.Conv2D(depth,3, activation='relu',padding="same")(x)
        
    for i in range(1,conv_by_block-1):
        x=layers.Conv2D(depth,3, activation='relu',padding="same")(x)
    
    x=layers.Conv2D(depth,3,padding="same")(x)
         
    x=skiped+x
    x=layers.ReLU()(x)
   
    return x

def get_resnet_backbone(input_shape,hparams):
    """
    """
    assert hparams["conv_by_block"]>=2
    inputs=layers.Input((32,32,3))
    x=layers.Conv2D(hparams["depth_first_convolution"],7,strides=(2,2),activation='relu',padding="same")(inputs)
    
    for block in range(1,hparams["number_of_block"]+1):
        x=apply_resnet_block(x,block in hparams["downsample_num"],hparams["conv_by_block"])
       
        
    x=layers.Conv2D(hparams["output_dim"],3, activation='relu',padding="same")(x)
    
    if hparams["globalPoolingType"]=="Mean":
        
        x=layers.GlobalAveragePooling2D()(x)
 
    return tf.keras.Model(inputs,x)



hparams_resnet={"depth_first_convolution":64,
                "output_dim":2048,
                "number_of_block":5,
                "downsample_num":[4,5],
                "conv_by_block":2,
                "globalPoolingType":"Mean"
            }




resnet=get_resnet_backbone(input_shape,hparams_resnet)

resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_12 (Conv2D)             (None, 16, 16, 64)   9472        ['input_6[0][0]']                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 16, 16, 64)   36928       ['conv2d_12[0][0]']              
                                                                                                  
 conv2d_14 (Conv2D)             (None, 16, 16, 64)   36928       ['conv2d_13[0][0]']              
                                                                                            

In [10]:
early_stop=tf.keras.callbacks.EarlyStopping(
    monitor="sparse_categorical_accuracy", patience=3, restore_best_weights=True
)


encoder = create_encoder(resnet)
classifier = create_classifier(encoder)
classifier.summary(expand_nested=False)

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop])

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Model: "cifar10-classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functional  (None, 2048)             6099719   
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                

In [11]:
early_stop_unsup=tf.keras.callbacks.EarlyStopping(
    monitor="loss", patience=1, restore_best_weights=True
)
learning_rate=0.001
resnet=get_resnet_backbone(input_shape,hparams_resnet)

encoder = create_encoder(resnet)

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop_unsup]
)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functional  (None, 2048)             6099719   
 )                                                               
                                                                 
 dense_4 (Dense)             (None, 128)               262272    
                                                                 
Total params: 6,361,991
Trainable params: 6,361,984
Non-trainable params: 7
_________________________________________________________________
Epoch 1/50
189/189 [==============================] - 39s 162ms/step - loss: 5.5790
Epoch 2/50
189/189 [==============================] - 31s 164ms/step - loss: 5.4526
Epoch 3/50
189/189 [==============================] 

In [15]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=1,callbacks=[early_stop])

history = classifier.fit(x=x_train, y=y_train,validation_data=test_dataset, batch_size=batch_size, epochs=num_epochs,callbacks=[early_stop])

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


189/189 [==============================] - 21s 83ms/step - loss: 31.9432 - sparse_categorical_accuracy: 0.7771 - val_loss: 3.4085 - val_sparse_categorical_accuracy: 0.7129
Epoch 1/50
189/189 [==============================] - 15s 77ms/step - loss: 2.4891 - sparse_categorical_accuracy: 0.7258 - val_loss: 1.4883 - val_sparse_categorical_accuracy: 0.7265
Epoch 2/50
189/189 [==============================] - 15s 78ms/step - loss: 1.3891 - sparse_categorical_accuracy: 0.7159 - val_loss: 1.2755 - val_sparse_categorical_accuracy: 0.7332
Epoch 3/50
189/189 [==============================] - 15s 77ms/step - loss: 1.1902 - sparse_categorical_accuracy: 0.7212 - val_loss: 1.1133 - val_sparse_categorical_accuracy: 0.7453
Epoch 4/50
189/189 [==============================] - 15s 77ms/step - loss: 1.1027 - sparse_categorical_accuracy: 0.7184 - val_loss: 1.1887 - val_sparse_categorical_accuracy: 0.7472
Epoch 5/50
189/189 [==============================] - 15s 77ms/step - loss: 1.0021 - sparse_categori